In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
sub=pd.read_csv('/kaggle/input/diabetic-retinopathy-classification-f1-score-4/kaggle_retina/submission.csv')
sub

,id_code,diagnosis
0,de4cdabbce6d,4
1,ba08cee68c71,4
2,537e50fdf22e,4
3,ab1c20a94f3f,4
4,3aa2b1ce6700,4
...,...,...
1460,43fb6eda9b97,4
1461,27933cdbe0cc,4
1462,3bf2deaa5ef0,4
1463,527bea76116c,4


In [4]:
data = pd.read_csv('/kaggle/input/diabetic-retinopathy-classification-f1-score-4/kaggle_retina/train.csv')
data

,id_code,diagnosis
0,c1e6fa1ad314,1
1,ed246ae1ed08,4
2,4d3de40ced3a,0
3,ae8472f8d310,1
4,3c28fd846b43,0
...,...,...
2192,87774aafe068,4
2193,1b4625877527,2
2194,bfe467b7e997,2
2195,d2cd47ed2c1d,0


In [8]:
data = pd.read_csv('/kaggle/input/diabetic-retinopathy-classification-f1-score-4/kaggle_retina/train.csv')
data['id_code'] = data['id_code']+'.png'
data['diagnosis'] = data['diagnosis'].astype(str)
data

,id_code,diagnosis
0,c1e6fa1ad314.png,1
1,ed246ae1ed08.png,4
2,4d3de40ced3a.png,0
3,ae8472f8d310.png,1
4,3c28fd846b43.png,0
...,...,...
2192,87774aafe068.png,4
2193,1b4625877527.png,2
2194,bfe467b7e997.png,2
2195,d2cd47ed2c1d.png,0


In [9]:
train_dir = '/kaggle/input/diabetic-retinopathy-classification-f1-score-4/kaggle_retina/train'
image_size = (224, 224)  # Adjust the size based on your requirements
batch_size = 32  # Adjust the batch size as needed

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize pixel values and split for validation

train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory=train_dir,
    x_col='id_code',
    y_col='diagnosis',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory=train_dir,
    x_col='id_code',
    y_col='diagnosis',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1758 validated image filenames belonging to 5 classes.
Found 439 validated image filenames belonging to 5 classes.


In [11]:
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # Adjust the number of output classes based on your problem
])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 26, 26, 128)     

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10  # Adjust the number of epochs as needed

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/10
54/54 [==============================] - 29s 358ms/step - loss: 0.9658 - accuracy: 0.6570 - val_loss: 0.7635 - val_accuracy: 0.7163
Epoch 2/10
54/54 [==============================] - 12s 222ms/step - loss: 0.7649 - accuracy: 0.7196 - val_loss: 0.7713 - val_accuracy: 0.7163
Epoch 3/10
54/54 [==============================] - 12s 229ms/step - loss: 0.7623 - accuracy: 0.7211 - val_loss: 0.7822 - val_accuracy: 0.6779
Epoch 4/10
54/54 [==============================] - 12s 225ms/step - loss: 0.7329 - accuracy: 0.7381 - val_loss: 0.8027 - val_accuracy: 0.7043
Epoch 5/10
54/54 [==============================] - 12s 221ms/step - loss: 0.7088 - accuracy: 0.7445 - val_loss: 0.7793 - val_accuracy: 0.6995
Epoch 6/10
54/54 [==============================] - 12s 225ms/step - loss: 0.6639 - accuracy: 0.7567 - val_loss: 0.7952 - val_accuracy: 0.7091
Epoch 7/10
54/54 [==============================] - 12s 228ms/step - loss: 0.6497 - accuracy: 0.7596 - val_loss: 0.7785 - val_accuracy: 0.7188

In [16]:
model.save('diabetic_retinopathy_classifier_model.h5')  # Replace with your desired model name and path

In [17]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
image_size = (224, 224)  # Adjust the size based on your requirements
test_dir = '/kaggle/input/diabetic-retinopathy-classification-f1-score-4/kaggle_retina/test'  # Replace with the path to your test directory

file_names = []
images = []

for file_name in os.listdir(test_dir):
    file_path = os.path.join(test_dir, file_name)
    image = load_img(file_path, target_size=image_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize the pixel values
    images.append(image)
    file_names.append(file_name)

images = np.array(images)

In [18]:
predictions = model.predict(images)
predicted_labels = np.argmax(predictions, axis=1)  # Get the index of the highest probability class

46/46 [==============================] - 1s 18ms/step


In [23]:
submission_data = {
    'id_code': file_names,
    'diagnosis': predicted_labels
}

submission_df = pd.DataFrame(submission_data)
submission_df


,id_code,diagnosis
0,ef476be214d4.png,0
1,17f6c7072f61.png,0
2,8d13c46e7d75.png,0
3,be68322c7223.png,2
4,88e4399d207c.png,0
...,...,...
1460,61e301bd3c25.png,2
1461,a8dea22ef903.png,0
1462,f72adcac5638.png,0
1463,86fbac86ed3e.png,1


In [26]:
submission_df['id_code']=submission_df['id_code'].str.replace('.png','')
submission_df

/tmp/ipykernel_28/1846239051.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  submission_df['id_code']=submission_df['id_code'].str.replace('.png','')


,id_code,diagnosis
0,ef476be214d4,0
1,17f6c7072f61,0
2,8d13c46e7d75,0
3,be68322c7223,2
4,88e4399d207c,0
...,...,...
1460,61e301bd3c25,2
1461,a8dea22ef903,0
1462,f72adcac5638,0
1463,86fbac86ed3e,1


In [27]:
submission_df.to_csv('submission_diabetic_retinopathy_img.csv', index=False)  # Replace with your desired submission file name and path